## Lab 9: Plotting Jet Streaks
<br /><br />

In this lab, we create plots of the full (or total) wind, ageostrophic wind, and divergence to help us analyze jet streaks and the vertical motion associated with jet streaks.  On the Python end of things, we continue to work to create cleaner code while also starting to work to figure out how to use modules on our own.
<br />

### Module Documentation

1. Xarray Dataset: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
2. Matplotlib Pyplot: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
3. Caropy crs: https://scitools.org.uk/cartopy/docs/latest/reference/crs.html
4. Cartopy Feature: https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html
5. Matplotlib Colors: https://matplotlib.org/stable/gallery/color/named_colors.html
6. Matplotlib Contour: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html
7. Matplotlib Barbs: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.barbs.html
8. Matplotlib Color Maps: https://matplotlib.org/stable/tutorials/colors/colormaps.html
9. Matplotlib Contourf: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html
10. Matplotlib Vectors: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html
11. MetPy Calculations: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html
12. Scipy Gaussian Filter: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html



If you have any questions about the code below, feel free to reach out to me at mpvossen@uwm.edu. I am always willing to further explain the code. <br /> <br />

---

<br />
1. As usual, we start by importing the modules we need for our Python code.

In [ ]:
#from the dates and time code(datetime), import the date and time reading capabilities (datetime)
from datetime import datetime

#import the module numpy and save it to np
import numpy as np

#import the cartopy (cartopy) module's coordinate reference system (.crs) and save it to the variable crs
import cartopy.crs as crs

#import the cartopy (cartopy) module's ability to plot geographic data (.feature) and save it to the variable cfeature 
import cartopy.feature as cfeature

#import the pyplot submodule from the matplotlib module
import matplotlib.pyplot as plt

#from the scipy module's ndimage submodule, import the function gaussian_filter
from scipy.ndimage import gaussian_filter

#import the module xarray and save it to xr
import xarray as xr

#from the metpy submodule units import the units function
from metpy.units import units

#import the calculation submodule form the metpy module and save it to the variable calc
import metpy.calc as calc

<br /><br />
2. Continuing the theme of previous labs, we create a function to process our data.  In the function below, we open a data file; select the desired isobaric level; limit the data to the United States; convert wind units (for barbs and contours only) to kt; and calculate divergence, wind speed, ageostrophic wind, and ageostrophic wind magnitude.  Watch out in the comments below for areas you need to complete.<br /><br /> For documentation on how to calculate divergence using MetPy, please visit https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.divergence.html.

In [ ]:
"""
Below, I define a function to retrieve and process GFS analysis upper-air data.  This function opens the GFS analysis data, retains only the desired pressure level, 
converts the wind units, calculates divergence, calculates the ageostrophic wind, and calculates and adds the wind speed to the dataset.

INPUT:
    level : INTEGER
        The level in hPa at which you want upper-air data.
    time : DATETIME
        The time at which you would like upper-air data.
    
OUTPUT:
    leveled_data : XARRAY DATASET
        The xarray containing your processed GFS analysis data.

"""


def process_upper_air_data(level, time):
    """
    Specify the location of the upper-air data on the JupyterHub.
    """
    lab_data_loc = "/data/AtmSci360/Lab_9/"
    
    """
    Open the GFS data using xarray.  Since the data are once again GRIB-formatted data, we can use xarray the same way we did in Lab 4.  Be sure to add filter_by_keys={'typeOfLevel': 'isobaricInhPa'}.
    """
    model_data = 
    
    """
    We only want data at a single isobaric level.  Limit the data in the xarray to only this specified level.
    """
    leveled_data = 
    
    
    """
    To save time on plotting, here I limit the dataset to only keep data that is between 20°N and 65°N.
    """
    leveled_data = leveled_data.sel(latitude = slice(65,20))
    
    
    """
    To save time on plotting, here I limit the dataset to only keep data that is between 145°W and 45°N.
    Note that GFS defines longitude from 0-360° rather than -180° to 180°, so we must convert
    western longitudes to their 0-360° equivalents.
    """
    leveled_data = leveled_data.sel(longitude = slice(360-145,360-45))
    
        
    """
    Calculate divergence and the ageostrophic wind.  The first step is to find the distance between each cell from it's neighbors using the 
    MetPy lat_lon_grid_deltas calculation function to find the x- and y-distances of each cell to it's neighbors.  This function only 
    requires the latitude and longitude data from the model.
    """
    dx, dy = calc.lat_lon_grid_deltas(leveled_data["longitude"], leveled_data["latitude"])
    
    """
    We are ready to calculate divergence.  Using the documentation link I gave you above, fill out the divergence
    function below.  Note that dx and dy are not optional arguments for this function.
    """
    div = calc.divergence()
    
    """
    To keep everything in one place, add the calculated divergence variable (div) to the xarray as the 
    variable divergence.
    """
    leveled_data = leveled_data.assign(divergence=div)
    
    """
    We will work with the ageostrophic wind in kt rather than m/s. This requires
    converting our units. Note that MetPy will recognize these units when the data
    are provided to the ageostrophic wind computing function below, such that the returned
    ageostrophic wind data will have units of kt.
    """
    leveled_data['u'] = leveled_data["u"].metpy.convert_units('kt')
    leveled_data['v'] = leveled_data["v"].metpy.convert_units('kt')
    
    """
    Now we are ready to calculate the ageostrophic wind.  The code is complete, but
    you should refer to the documentation to see how it works.  
    
    Documentation: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.ageostrophic_wind.html#metpy.calc.ageostrophic_wind
    """
    u_ageo, v_ageo = calc.ageostrophic_wind(leveled_data["gh"], leveled_data["u"], leveled_data["v"], dx=dx, dy=dy, latitude=leveled_data["latitude"])
    
    """
    Assign the ageostrophic components to the xarray dataset.  This keeps everything in one place and your code stays clean.
    assign(variable_name=variable_data)    
    """
    leveled_data = leveled_data.assign(u_ageostrophic=u_ageo)
    leveled_data = leveled_data.assign(v_ageostrophic=v_ageo)
    
    """
    It will be useful to have the ageostrophic wind magnitude for later.  In the lines below, calculate the 
    ageostrophic wind magnitude and save it back to your xarray dataset.
    """
    
    
    
    """
    We will also need the wind speed (for the full wind).  In the lines below, calculate the wind speed
    and save it back to your xarray dataset.
    """
    
    
    
    """
    Finally, our data processing is complete and so we need to return the processed data.
    In the line below, add a line of code to return the processed model data.
    """
    



<br /><br />
3. To start our analysis, we plot a straight jet streak that occurred on November 8th, 2021 at 1200 UTC.  Call your data processing function to get data for November 8th, 2021 at 1200 UTC at jet level (300 hPa).

<br /><br />
4. We wish to create three plots, one each for the full wind, ageostrophic wind, and divergence.  All three of these plots have a common basis, so we can create the base plot in a seperate function that can be called by subsequent plotting functions for each variable.  Follow the comments in the code below to complete the function.

In [ ]:
"""
Below, I define a function to initialize a plot over the United States.  This function creates the plot, sets up the projection, 
sets the extent, adds the desired geographic data, and plots isohypses.

INPUT:
    data : XARRAY DATASET
        The GFS data to use to plot isohypses.

OUTPUT:
    ax : MATPLOTLIB AXES
        The initialized plot.
"""


def initialize_plot(data):
    """
    Setup a Lambert Conic Conformal Projection centered at 35°N and 95°W.  Have the cone of the Lambert Conic Conformal projection intersect the Earth at 27.5°N and 42.5°N.
    """
    
        
    """
    Create a figure with a size of 1150px x 700px and a resolution of 300 dots per inch,
    then set up an axes object (named ax) using the projection we previously defined for our map.
    """

    
    """
    Add an appropriate amount of geographic data.  Be sure you follow "good map" suggestions with the geographic data styling.
    """

    
    """
    Limit the map to between 125°-65°W longitude and 23°-60°N latitude.
    """
    
    
    """
    Add isohypses.  As with previous labs, first smooth the geopotential height data to make them easier to analyze. 
    Set up the Gaussian filter for the geopotential heights ('gh'). Choose an appropriate smoothing value for these data.
    """
    smooth_heights = 
    
    """
    We can now isopleth the smoothed geopotential height data.  In the line below, create line contours of geopotential height.
    """
    cont_h = 
    
    """
    Every contour plot needs labels for each contour.  In the line below, add labels to your line contours.
    """
    
    """
    You can ignore this next section, which labels some points (A-D) that are referenced later in the lab.
    """
    
    if str(data.time.values) == "2021-11-08T12:00:00.000000000": 
        ax.text(-105,48,"A", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-103,45,"B", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-90,51,"C", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-88.5,47,"D", size=12, weight="bold", transform=crs.PlateCarree())
    else:
        ax.text(-97,47,"A", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-92,47,"B", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-78,43,"C", size=12, weight="bold", transform=crs.PlateCarree())
        ax.text(-72,43,"D", size=12, weight="bold", transform=crs.PlateCarree())
    
    
    """
    Finally, have the function return our axes object to the calling function.
    """
    
    

<br /><br />
5.  With the base plot created, we are now ready to create each of our analyses.  Let's start by creating a full wind plot.  Follow along with the comments and fill in where you are asked.

In [ ]:
"""
Below, I define a function to plot upper-air full wind data.  This function adds color-filled 
isotachs and wind barbs for the full wind.

INPUT:
    time : DATETIME
        The date and time for which the plot is valid.
    level : INTEGER
        The level at which the plot is valid.
    data : XARRAY DATASET
        The GFS analysis data.

    
"""

def plot_winds(time, level, data):
    """
    Call the function to create the base plot.  Have it save the output to the variable ax.
    """

              
    """
    Start by applying an appropriate Gaussian filter to the wind speed data.
    """
    smooth_wind = 
              
    """
    Create color-filled contours of the smoothed wind.  Be sure to style the contours so they are easy to analyze.  Use
    the 'extend = "max"' option to tell the contour statement to contour values that exceed the contour range.
    """
    cont =
    
              
    """
    Next, create a color bar with appropriate styling information for the color-filled contours.
    """
    
    
    
    """
    Add wind barbs with a spacing of 12 grid points.
    Add appropriate styling information to make the plot easier to understand.
    """
    

    
    """
    Finally, add an appropriate title for the map that shows what is plotted and the time at which the map is valid.
    """
    

<br /><br />
6. Call the plotting function you just created.

<br /><br />
7. We also wish to plot the ageostrophic wind.  In the code block below, create a function to plot the ageostrophic wind following the instructions in the comments.

In [ ]:
"""
Below, I define a function to plot upper-air ageostrophic wind data.  This function plots color-filled
ageostrophic wind magnitude and wind vectors for the ageostrophic wind.

INPUT:
    time : DATETIME
        The date and time at which the plot is valid.
    level : INTEGER
        The level at which the plot is valid.
    data : XARRAY DATASET
        The GFS analysis data.

    
"""
def plot_ageo_wind(time, level, data):
    """
    Call the function to create the base plot.  Have it save the output to the variable ax.
    """

    """
    Start by applying an appropriate Gaussian filter to the ageostrophic wind magnitude.
    """
    smooth_wind = 
                        
    """
    In the line below, create color-filled contours of the smoothed ageostrophic wind.  Be sure to style the contours so they are easy to analyze.  Use
    the 'extend = "max"' option to tell the contour statement to contour values that exceed the contour range.
    """
    cont =         
              
    """
    Next, create a color bar with appropriate styling information for the color-filled contours.
    """
    
    
    """
    Now, plot the ageostrophic wind vectors.  The first step is to determine how to space the data.  There are data points every 0.25° latitude
    and longitude, which is too frequent to be able to draw legible vectors.  Instead, we can space the vectors by skipping a specific number of
    data points.  In the case below, we retain data only every 12 points (or ~250 km apart).  
    """
    wind_slice_x = slice(None, None, 12)
    wind_slice_y = slice(None, None, 12)
            
    """
    To make the data easier to interpret, apply a Gaussian filter to the u and v ageostrophic wind components.
    """
    u_smooth = gaussian_filter(data["u_ageostrophic"].values, 2)
    v_smooth = gaussian_filter(data["v_ageostrophic"].values, 2)                  
    
    
    """
    Create a temporally varying scale vector length. 
    """
    if time.day == 8:
        scale = 100   
    else:
        scale = 175
    
    
    """
    Below is the matplotlib command to plot vectors.  The first and second arguments are the x (longitude) and y (latitude) values, respectively.  
    Note that latitude and longitude are 1-D whereas the ageostrophic wind variables are 2-D.  Thus, for position data, we only need to use the wind slice direction that is appropriate.
    The third and fourth arguments are the u- and v-ageostrophic wind components.  We need to use both wind slices since our data is 2-D.
    The fifth argument (color='black') determines the vector colors.  The matplotlib colors link can once again be used to see what colors you can use to plot the vectors.
    The sixth argument says what horizonal coordinate system the data are in.  
    The seventh argument (linewidth=0.6) determines the vector's width.  Larger values make them thicker and smaller values make them thinner.
    The eighth argument (scale_units="inches) units for the vector length.  It is best to use inches.
    The last argument is the scaling for the vectors.  Larger values make the vectors decrease in length, smaller values make the vectors increase in length.
    """
    q = ax.quiver(data["longitude"][wind_slice_x].values, data["latitude"][wind_slice_y].values,
             u_smooth[wind_slice_x, wind_slice_y],
             v_smooth[wind_slice_x, wind_slice_y],
             color='black',transform=crs.PlateCarree(),linewidth=0.6, scale_units='inches', scale=scale)
    
    """
    Next, we need to add a reference vector to connect vector length to speed. 
    In the code below, a reference vector is added above our map on the right side.
    
    Argument one is the vector plot object from the code above.
    Arguments two and three are the reference vector's x and y positions, respectively.  As elaborated on below,
      (1,1) represents the upper right corner of the map; values greater than 1 extend beyond the map.
    Argument four is the magnitude represented by the reference vector in the same units as the data.
    Argument five is the vector's desired label.
    Argument six is the label's position relative to the reference vector.  In this case, the label will
      plot to the right (or east) of the vector.
    Argument seven is the coordinate system that we are using in our x and y position to place our key.
    Here, we use "axes" so we cam easily position the key in the plot by using values between 0 (far left/bottom) and 1 (far right/top).
    The last argument is the font properties.  Here we can add font sizing, font weight, and font type
      arguments to the label.
    """
    ax.quiverkey(q, 1.09, 1.03, 25.,'25 kt', labelpos='E', coordinates='axes',
                     fontproperties={'size':10})
              
    """
    Finally, add an appropriate title for the map that shows what is plotted and the time at which the map is valid.
    """
    
    


<br /><br />
8. Call the ageostrophic wind plotting function.

<br /><br />
9. Finally, to complete our analyses, we wish to plot divergence.  In the function below, create a function to plot the divergence following the instructions in the comments.

In [ ]:
"""
Below, I define a function to plot divergence.  This function plots color-filled isopleths 
for divergence and wind barbs for the full wind.

INPUT:
    time : DATETIME
        The date and time at which the plot is valid.
    level : INTEGER
        The level at which the plot is valid.
    data : XARRAY DATASET
        The GFS analysis data.



"""
def plot_divergence(time, level, data):
    """
    Call the function to create the base plot.  Have it save the output to the variable ax.
    """
    
    
         
              
    """
    Start by applying an appropriate Gaussian filter to the divergence data.
    After doing so, multiply the divergence data by 10^-5 so it is easier to analyze.  
    """
    smooth_div = 
    
    """
    In the line below, create color-filled contours of the smoothed divergence.  Be sure to style the contours so they are easy to analyze.  Use
    the 'extend = "both"' option to tell the contour statement to contour values that are greater and smaller than the specified contour range.
    
    Hint: A good contour range for divergence is between -10 * 10^-5 and 10 * 10^-5.
    """
    
    
    
    """
    Next, create a color bar with appropriate styling information for the color-filled contours.
    """
    
       
    """
    Add wind barbs to your plot with a spacing of 12 grid points between barbs.
    Add appropriate styling information to make the plot easier to understand.
    """
    
    
    
    """
    Finally, add an appropriate title for the map that shows what is plotted and the time at which the map is valid.
    """
    
    

<br /><br />
10. Call the divergence plotting function.

<br /><br />
11. We also wish to analyze curved jet streaks.  Re-run your functions to create plots for the curved jet streak that occurred on October 18th, 2022 at 0600 UTC.

### You have now completed the Python portion of the lab.  Be sure to submit the fully rendered Jupyter Notebook on GitHub when you are finished.